# Beers - Solved

In [ ]:
# Import findspark
import findspark

# Configure the environment
findspark.init()

# Import the Spark components required for the session creation
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Configure and create the session
conf = SparkConf()
conf = conf.setAppName('mds-session')
conf = conf.setMaster('local[*]')
spark = SparkSession.builder.config(conf = conf).getOrCreate()

In [ ]:
beers = spark.read.options(sep=',', header=False, inferSchema=True).csv('./data/beers.csv')
beers = beers.toDF('row_id', 'abv', 'ibu', 'id', 'name', 'style', 'brewery_id', 'ounces')
beers.show(5)

beers.createOrReplaceTempView('beers')

In [ ]:
breweries = spark.read.options(sep=',', header=False, inferSchema=True).csv('./data/breweries.csv')
breweries = breweries.toDF('id', 'name', 'city', 'state')
breweries.show(5)

breweries.createOrReplaceTempView('breweries')

## 1. ¿Cuántas filas hay en el set de datos de cervezas? ¿Y en el de cervecerías?

In [ ]:
beers.count()

In [ ]:
breweries.count()

## 2. ¿Cuál es el nombre de la cerveza que tiene más alcohol? Devolver sólo un registro y sólo con el nombre.

In [ ]:
beers.orderBy(beers.abv.desc()).select('name').first()['name']

In [ ]:
spark.sql('''
    SELECT name
    FROM beers
    ORDER BY abv DESC
    LIMIT 1
''').show()

## 3. ¿Cuántas cervezas sin alcohol (con nivel de alcohol a 0) hay en el set de datos?

In [ ]:
beers.where(beers.abv.isNull()).count()

In [ ]:
spark.sql('''
    SELECT count(*)
    FROM beers
    WHERE abv IS NULL
''').show()

## 4. ¿Cuál es el peso medio calculado sobre todas las cervezas?

In [ ]:
spark.sql('''
    SELECT AVG(ounces)
    FROM beers
''').show()

In [ ]:

type(beers)

## 5. ¿Cuál es el el máximo nivel de alcohol por tipo de cerveza? Ordena los resultados por nivel de alcohol de mayor a menor.

In [ ]:
max_abvs = beers.groupBy('style').agg(F.max('abv').alias('abv'))
max_abvs.orderBy(max_abvs.abv.desc()).show(5)

In [ ]:
spark.sql('''
    SELECT style, MAX(abv) AS abv
    FROM beers
    GROUP BY style
''').show()

## 6. ¿Cuál es el nombre de la cervecería con la cerveza menos amarga? Devolver sólo un registro y sólo con el nombre.

In [ ]:
beers = beers.withColumnRenamed('name', 'beer_name')
breweries = breweries.withColumnRenamed('id', 'brewery_id')
data = beers.join(breweries, on='brewery_id')

data.select('name', 'ibu').distinct().orderBy(data.ibu.asc_nulls_last()).show(5)

In [ ]:
spark.sql('''
    SELECT breweries.name, MIN(ibu) as brewery_ibu
    FROM beers, breweries
    WHERE beers.brewery_id = breweries.id
    GROUP BY breweries.name
    ORDER BY brewery_ibu ASC NULLS LAST
    LIMIT 1
''').show()

## 7. ¿Cuál es la cerveza más amarga de Chicago? Devolver sólo un registro y sólo con el nombre.

In [ ]:
data.where(data.city == 'Chicago').orderBy(data.ibu.desc()).select('beer_name').show(1)

In [ ]:
spark.sql('''
    SELECT beers.name
    FROM beers, breweries
    WHERE beers.brewery_id = breweries.id
      AND city = 'Chicago'
    ORDER BY ibu DESC
    LIMIT 1
''').show()

## 8 .¿Cuál es el estado que tiene más cervezas y cuántas tiene? Devolver sólo un registro y sólo con las siglas.

In [ ]:
breweries = breweries.withColumn('state', F.ltrim('state'))

data = beers.join(breweries, beers.brewery_id == breweries.id)
data = data.groupBy('state').count()
data.orderBy(data['count'].desc()).show(5)

In [ ]:
spark.sql('''
    SELECT state, COUNT(*) as cervezas
    FROM beers, breweries
    WHERE beers.brewery_id = breweries.id
    GROUP BY state
    ORDER BY cervezas DESC
    LIMIT 1
''').show()

## 9. ¿Cuál es la desviación estándar de nivel de alcohol, amargor y peso por ciudad y tipo de cerveza? Ordena los resultados por ciudad y tipo de cerveza ascendente.

In [ ]:
data = beers.join(breweries, beers.brewery_id == breweries.id)

data.groupBy(data.city, data.style).agg(
    F.stddev(data.abv).alias('abv'),
    F.stddev(data.ibu).alias('ibu'),
    F.stddev(data.ounces).alias('ounces'),
).orderBy(['city', 'style'], ascending=[True, True]).show()

## 10. ¿Qué ciudad vende la cerveza con el nombre más largo? Devuelve sólo un registro con el nombre de la ciudad, el nombre de la cerveza y la longitud del nombre.

In [ ]:
data.select('city', 'beer_name', F.length('beer_name').alias('total'))
).orderBy('total', ascending=False).show(1)

In [ ]:
spark.sql('''
    SELECT city, beers.name, LENGTH(beers.name)
    FROM beers, breweries
    where beers.brewery_id = breweries.id
    ORDER BY LENGTH(beers.name) DESC
    LIMIT 1
''').show()